In [1]:
import pandas as pd
import numpy as np
import nltk
import networkx as nx
from networkx.exception import PowerIterationFailedConvergence
from rouge import Rouge
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english'))
from nltk.tokenize import word_tokenize
import re
import nltk.data
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
import string

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
rouge_=Rouge()
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /home/sunild/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sunild/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
"""
FIXED VARIABLES DECLARED
"""
EMBEDDING_SIZE=100 # OPTIONS: 50,100,200,300
SENT_SUMMARY_COUNT=5
SENT_SUMMARY_THERSHOLD=0.35
HI_DATA_PATH='../data/hi-data/test.csv' # BOTH ARTICLES AND SUMMARIES
EN_DATA_PATH_SRC='../data/preprocessed_truncated/test.txt.src.tokenized.fixed.cleaned.final.truncated.txt' # ONLY ARTICLES
EN_DATA_PATH_TGT='../data/preprocessed_truncated/test.txt.tgt.tokenized.fixed.cleaned.final.truncated.txt' # ONLY SUMMARIES
HI_EMBEDDINGS_PATH='../data/glove/hi/hi-d100-glove.txt'
EN_EMBEDDINGS_PATH='../data/glove/en/glove.6B.100d.txt'

In [3]:
"""
HELPER FUNCTIONS FOR MAIN SUMMARY GENERATION FUNC
"""
def cosine_similarity(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))


def load_glove_model(file_path):
    """
    function to load glove embeddings from txt file into a dict
    """
    glove_model = {}
    with open(file_path,'r',encoding="utf-8",errors='ignore') as f:
        try:
            for i,line in enumerate(f):
                # if i%10000==0:
                #     print(i)
                split_line = line.split()
                word = split_line[0]
                embedding = np.array(split_line[-EMBEDDING_SIZE:], dtype=np.float64)
                glove_model[word] = embedding
        except Exception as e:
            print(e)
            print(line,i)
    print(f"total {len(glove_model)} words loaded from glove model, each of size {EMBEDDING_SIZE}.")
    return glove_model

# load glove embeddings for both hindi and english
glove_embeddings=load_glove_model(file_path=EN_EMBEDDINGS_PATH)
hi_glove_embeddings=load_glove_model(file_path=HI_EMBEDDINGS_PATH)
# save all glove embeddings in a single dict
# this takes care of both languages that are present in hindi texts; although it makes overall process a bit slow
glove_embeddings.update(hi_glove_embeddings)    

total 400000 words loaded from glove model, each of size 100.
total 1732951 words loaded from glove model, each of size 100.


In [4]:
stop_words = set(stopwords.words('english'))
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

<>:2: DeprecationWarning: invalid escape sequence \,


In [5]:
documents=["gop eyes gains as voters in 11 states pick governors enlarge this image toggle caption jim cole/ap jim cole/ap voters in 11 states will pick their governors tonight , and republicans appear on track to increase their numbers by at least one , with the potential to extend their hold to more than two-thirds of the nation ' s top state offices . eight of the gubernatorial seats up for grabs are now held by democrats ; three are in republican hands . republicans currently hold 29 governorships , democrats have 20 , and rhode island ' s gov . lincoln chafee is an independent . polls and race analysts suggest that only three of tonight ' s contests are considered competitive , all in states where incumbent democratic governors aren ' t running again : montana , new hampshire and washington . while those state races remain too close to call , republicans are expected to wrest the north carolina governorship from democratic control , and to easily win gop-held seats in utah , north dakota and indiana . democrats are likely to hold on to their seats in west virginia and missouri , and are expected to notch safe wins in races for seats they hold in vermont and delaware . holding sway on health care while the occupant of the governor ' s office is historically far less important than the party that controls the state legislature , top state officials in coming years are expected to story_separator_special_tag gop eyes gains as voters in 11 states pick governors jim cole / ap i jim cole / ap voters in 11 states will pick their governors tonight , and republicans appear on track to increase their numbers by at least one , and with the potential to extend their hold to more than two-thirds of the nation ' s top state offices . eight of the gubernatorial seats up for grabs today are now held by democrats ; three are in republican hands . republicans currently hold 29 governorships , democrats have 20 ; and rhode island ' s gov . lincoln chafee is an independent . polls and race analysts suggest that only three of tonight ' s contests are considered competitive , all in states where incumbent democratic governors aren ' t running again : montana , new hampshire and washington . while those state races remain too close to call , republicans are expected to wrest the north carolina governorship from democratic control , and to easily win gop-held seats in utah , north dakota and indiana . democrats are likely hold on to their seats in west virginia and missouri ; and expected to notch safe wins in races for seats they hold in vermont and delaware . holding sway on health care while the occupant of the governor ' s office is historically far less important than the party that controls the state legislature , top state officials in coming years are expected to"]

In [6]:
def get_sent_vect(probe_text):
    """
    func to return sent vectors
    """

    break_sentences=tokenizer.tokenize(probe_text)
    clean_sentences=[]
    for item in break_sentences:
      item=item.translate(str.maketrans('', '', string.punctuation))
      item=' '.join( [w for w in item.split() if len(w)>1] )
      words = word_tokenize(item)
      words = [word for word in words if len(words)>1]
      words = [word.lower() for word in words]
      words = [word for word in words if word not in stop_words]
      item=' '.join(words)    
      clean_sentences.append(item)
    cln_summ=''
    cln_summ = '.'.join(clean_sentences)
    # print(cln_summ)
    #print(cln_summ)
    tfidf = CountVectorizer()
    response = tfidf.fit_transform([cln_summ])
    #print(response)
    unique_word_list = tfidf.get_feature_names_out()
    tf_idf={}
    for col in response.nonzero()[1]:
      tf_idf[unique_word_list[col]]= response[0, col]
    svd_matrix=np.zeros([len(unique_word_list),len(clean_sentences)])
    for i in range(0,len(clean_sentences)):
      sen=clean_sentences[i].split()
      for val in sen:
        #print(val)
        index=np.where(unique_word_list == val)
        index_length=len(index[0])
        if(index_length == 1):
          svd_matrix[index,i]=tf_idf[val]
    #print(svd_matrix)
    u,sigma,v_trans=np.linalg.svd(svd_matrix,full_matrices=False,compute_uv=True)
    sum_sigma=np.sum(sigma)
    iter_sum=0
    check_coverage=0
    for i in range(0,len(sigma)):
      iter_sum=iter_sum+sigma[i]
      percent_contribution=iter_sum/sum_sigma
      #print("the percent contributuin is ",percent_contribution)
      if(percent_contribution<0.70):
        check_coverage=i+1
    u_trimmed=u[:,:check_coverage]
    v_trimmed=v_trans[:check_coverage,:]

    sent_vects=[]
    for i,sent in enumerate(clean_sentences):
        # sent_words=sent.split(' ')
        tmp_=0
        for k,concept_weight in enumerate(v_trimmed[:,i]):
            foo_=0
            for j,word in enumerate(unique_word_list):
                if word in glove_embeddings:
                    # print('it is')
                    foo_+=u_trimmed[j,k]*glove_embeddings[word]
            tmp_+=np.abs(concept_weight)*foo_
        
        sent_vects.append(tmp_)
      
    return sent_vects

In [7]:
def generate_summary(article_text):
    """
    function to get summary of article text
    """
    sent_tokens=sent_tokenize(article_text)
    try:
        sent_vects=get_sent_vect(article_text)
    except ValueError:
        return 0
    total_sents=len(sent_vects)
    similarity_mat=np.zeros((total_sents,total_sents))
    
    for i in range(total_sents):
        for j in range(total_sents):
            # this way diag entries will be zeros
            if i!=j:
                similarity_mat[i][j]=cosine_similarity(sent_vects[i],sent_vects[j])

    # create graph from similarity matrix
    network_graph=nx.from_numpy_array(similarity_mat)
    """
    apply pagerank algo to get scores
    """
    scores=nx.pagerank(network_graph, alpha=0.85, max_iter=1000)
    sorted_scores = sorted(((scores[i],sent_token) for i,sent_token in enumerate(sent_tokens)), reverse=True)

    # TODO: implement thershold version
    # in case some article does not have enough sentences
    if total_sents > SENT_SUMMARY_COUNT:
        summary_text=' '.join([sorted_scores[i][1] for i in range(SENT_SUMMARY_COUNT)])
    else:
        summary_text=' '.join([sorted_scores[i][1] for i in range(total_sents)])

    return summary_text


In [8]:
test_text='''
The National Cricket Academy is organising a camp in Bengaluru to help players from the north eastern states and those in the plate group upgrade their skill-sets under the tutelage of renowned coaches including Ashes winning bowling coach Troy Cooley. Close to 150 cricketers from the region are taking part in the camp and they will also get to interact with NCA head and India great VVS Laxman.The BCCI wants to ensure that players from Mizoram, Manipur, Meghalaya, Arunachal Pradesh, Sikkim get the best coaching and infrastructural facilities to be on par with the best domestic teams.
"This initiative by BCCI will give young and budding talents equal opportunity to hone their skills and represent India across formats. NE has enormous potential in sports and the board will keep no stone unturned to ensure that we are able to tap the best talent for the country," BCCI secretary Jay Shah tweeted.The fast bowlers are being trained by one of the world's acclaimed pace bowling coach Cooley while batters are working under the guidance of U-19 World Cup winning coach Hrishikesh Kanitkar.The spinners are under former India leg-spinner Sairaj Bahutule.
While BCCI hasn't provided any details about the duration of the camp, it is understood that the NCA has lined up a number of coaching programmes including U-19 Zonal Cricket Academy camps which will be held across various cities including Surat, Vijaywada to name a few.
Americans eat more ice cream than anyone else in the world-about 35 pints per person per year. From classic vanilla to green tea to salted caramel, the flavors are endless. But flavor isn’t this frozen dessert’s only attraction. Texture provides most of the sensual appeal of ice cream, and it’s maddeningly ephemeral: firm on the spoon yet creamy in the mouth as it melts. Understanding how to achieve this delicious juxtaposition is the key to making high-quality ice cream at home.
Sun-kissed beaches, crystal clear water, shimmering sand, cool breeze and scrumptious seafood define beach holidays, the best. Beach destinations guarantee a fun-packed vacation. If you’re an adventure enthusiast, you can indulge in watersports like scuba diving, snorkelling, sea walking and surfing. Treat your eyes to the exotic marine life which includes Clownfish, Mandarinfish, Mimic Octopus, Manta Rays, turtles and many more. Honeymooners can enjoy a romantic candle-light dinner, pamper themselves with a spa therapy and soak in the breathtaking sight of the sunset. Foodies can tantalise their palate with delectable prawns, crabs, lobsters and oysters. Photographers can try their hand at underwater photography and elevate their creative skills. If you’re planing on a relaxing vacation, beach destinations are perfect for you. Let your guard down, read a novel whilst resting in the hammock and rejuvenate your senses by booking a luxurious accommodation. Nature lovers can find inspiration in the splendid beauty of the beaches and changing hues of the sky. Solo travellers are bound to find solitude and experience spiritual bliss.
'''
generate_summary(test_text)

/home/sunild/anaconda3/envs/ir/lib/python3.9/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:108: DeprecationWarning: networkx.pagerank_scipy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  return pagerank_scipy(


'But flavor isn’t this frozen dessert’s only attraction. From classic vanilla to green tea to salted caramel, the flavors are endless. Nature lovers can find inspiration in the splendid beauty of the beaches and changing hues of the sky. "This initiative by BCCI will give young and budding talents equal opportunity to hone their skills and represent India across formats. If you’re an adventure enthusiast, you can indulge in watersports like scuba diving, snorkelling, sea walking and surfing.'

In [9]:
hi_data=pd.read_csv('../data/hi-data/test.csv',lineterminator='\n')
hi_data.drop('headline',axis=1,inplace=True)
hi_data.dropna(inplace=True)
hi_data.reset_index(drop=True,inplace=True)

In [10]:

"""
calculate summaries for hindi test dataset
"""
article_summary_list_hi=[]
bad_sent_vect=[]
bad_articles_hi=[]
for i,line in enumerate(list(hi_data['article'])):
    if i % 100 == 0:
        print(i)
        print("------")
    try:
        foo_=generate_summary(line.strip())
        if foo_==0:
            bad_sent_vect.append(i)
        else:
            article_summary_list_hi.append(foo_)
    except PowerIterationFailedConvergence:
        bad_articles_hi.append(i)
        continue

with open(f'../data/hi_article_summary_{EMBEDDING_SIZE}.txt','w') as file:
    file.write('\n'.join(article_summary_list_hi))

0
------


/tmp/ipykernel_17536/3958700682.py:5: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
/home/sunild/anaconda3/envs/ir/lib/python3.9/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:108: DeprecationWarning: networkx.pagerank_scipy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  return pagerank_scipy(
/home/sunild/anaconda3/envs/ir/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/sunild/anaconda3/envs/ir/lib/python3.9/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:505: RuntimeWarning: overflow encountered in subtract
  err = np.absolute(x - xlast).sum()
/tmp/ipykernel_17536/3958700682.py:5: RuntimeWarning: invalid value encountered in true_divide
  return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
/home/sunild/anaconda3/envs

100
------
200
------
300
------
400
------
500
------
600
------
700
------
800
------
900
------
1000
------
1100
------
1200
------
1300
------
1400
------
1500
------
1600
------
1700
------
1800
------
1900
------


In [11]:
bad_articles_hi.extend(bad_sent_vect)

In [12]:

"""
remove bad articles that does not have sufficient text data 
"""
foo_refs=list(hi_data['summary'])
hi_refs=[]
for i,article in enumerate(foo_refs):
    if i not in bad_articles_hi:
        hi_refs.append(article)

In [13]:

"""
calculate rouge score for hindi test dataset
"""
rouge_=Rouge()
rouge_scores=[]
bad_rouges=[]
for i,_ in enumerate(hi_refs):
    
    if i % 100 == 0:
        print(i)
        print("------")
    if i not in bad_articles_hi:
        try:
            rouge_scores.append(rouge_.get_scores(hyps=article_summary_list_hi[i],refs=hi_refs[i]))
        except RecursionError:
            bad_rouges.append(i)
            continue

0
------
100
------
200
------
300
------
400
------
500
------
600
------
700
------
800
------
900
------
1000
------
1100
------


In [14]:

rouge_1=[]
for score in rouge_scores:
    rouge_1.append(score[0]['rouge-1']['f'])
print('avg f1 score for rouge-1 measure is :',sum(rouge_1)/len(rouge_1))


rouge_1=[]
for score in rouge_scores:
    rouge_1.append(score[0]['rouge-1']['p'])
print('avg precision for rouge-1 measure is :',sum(rouge_1)/len(rouge_1))


rouge_1=[]
for score in rouge_scores:
    rouge_1.append(score[0]['rouge-1']['r'])
print('avg recall for rouge-1 measure is :',sum(rouge_1)/len(rouge_1))

avg f1 score for rouge-1 measure is : 0.32649549169141806
avg precision for rouge-1 measure is : 0.2137251092789283
avg recall for rouge-1 measure is : 0.8083749293421163


In [15]:

"""
calculate summaries for english test dataset
"""
article_summary_list=[]
bad_articles=[]
with open(EN_DATA_PATH_SRC,'r') as file:
    for i,line in enumerate(file):
        if i % 100 == 0:
            print(i)
            print("------")
        try:
            article_summary_list.append(generate_summary(line.strip()))
        except PowerIterationFailedConvergence:
            bad_articles.append(i)
            continue

0
------


/tmp/ipykernel_17536/3958700682.py:5: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))


100
------
200
------
300
------
400
------
500
------


/tmp/ipykernel_17536/3958700682.py:5: RuntimeWarning: invalid value encountered in true_divide
  return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))


600
------
700
------
800
------
900
------
1000
------
1100
------
1200
------
1300
------
1400
------
1500
------
1600
------
1700
------
1800
------
1900
------


In [16]:

with open(f'../data/en_article_summary_{EMBEDDING_SIZE}.txt','w') as file:
    file.write('\n'.join(article_summary_list))
ref_summary=[]
with open(EN_DATA_PATH_TGT,'r') as file:
    for i,line in enumerate(file):
        if i not in bad_articles:
            ref_summary.append(line[2:].strip())

In [17]:
# takes about 3 mins
rouge_scores_en=rouge_.get_scores(hyps=article_summary_list,refs=ref_summary,avg=True)
rouge_scores_en

{'rouge-1': {'r': 0.3017069519477892,
  'p': 0.436289026812864,
  'f': 0.34718193093084565},
 'rouge-2': {'r': 0.09827712184621379,
  'p': 0.15357259830846484,
  'f': 0.11533621185813975},
 'rouge-l': {'r': 0.26159781102530294,
  'p': 0.37955952839673435,
  'f': 0.30143751806527797}}